# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

M. Gárate  ->  M. Gárate  |  ['M. Gárate']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
H. Beuther  ->  H. Beuther  |  ['H. Beuther']


S. Li  ->  S. Li  |  ['S. Li']
Arxiv has 84 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2309.08752


extracting tarball to tmp_2309.08752...

 done.


M. Gárate  ->  M. Gárate  |  ['M. Gárate']


...

✔ → 21489:\section{Results and analysis}
  ↳ 46381:\section{Discussion}


✔ → 46381:\section{Discussion}
  ↳ 65040:\section{Summary and conclusions}
✔ → 65040:\section{Summary and conclusions}
  ↳ 68854:\begin{appendix}
✔ → 68854:\begin{appendix}
  ↳ 68871:\section{Integrated intensity contour maps}


✔ → 68871:\section{Integrated intensity contour maps}
  ↳ 71533:\section{Line profiles in detected molecules}
✔ → 71533:\section{Line profiles in detected molecules}
  ↳ 72850:\section{Correlation plots}
✔ → 72850:\section{Correlation plots}
  ↳ 74872:end


H. Beuther  ->  H. Beuther  |  ['H. Beuther']
S. Li  ->  S. Li  |  ['S. Li']


Found 98 bibliographic references in tmp_2309.09687/main.bbl.
syntax error in line 686: premature end of file


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.08752-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.08752) | **Millimeter emission in photoevaporating disks is determined by early  substructures**  |
|| <mark>M. Gárate</mark>, et al. |
|*Appeared on*| *2023-09-19*|
|*Comments*| *Accepted for publication in A&A*|
|**Abstract**| [abridged]Photoevaporation and dust-trapping are individually considered to be important mechanisms in the evolution and morphology of protoplanetary disks. We studied how the presence of early substructures affects the evolution of the dust distribution and flux in the millimeter continuum of disks that are undergoing photoevaporative dispersal. We also tested if the predicted properties resemble those observed in the population of transition disks. We used the numerical code Dustpy to simulate disk evolution considering gas accretion, dust growth, dust-trapping at substructures, and mass loss due to X-ray and EUV (XEUV) photoevaporation and dust entrainment. Then, we compared how the dust mass and millimeter flux evolve for different disk models. We find that, during photoevaporative dispersal, disks with primordial substructures retain more dust and are brighter in the millimeter continuum than disks without early substructures, regardless of the photoevaporative cavity size. Once the photoevaporative cavity opens, the estimated fluxes for the disk models that are initially structured are comparable to those found in the bright transition disk population ($F_\textrm{mm} > 30\, \textrm{mJy}$), while the disk models that are initially smooth have fluxes comparable to the transition disks from the faint population ($F_\textrm{mm} < 30\, \textrm{mJy}$), suggesting a link between each model and population. Our models indicate that the efficiency of the dust trapping determines the millimeter flux of the disk, while the gas loss due to photoevaporation controls the formation and expansion of a cavity, decoupling the mechanisms responsible for each feature. In consequence, even a planet with a mass comparable to Saturn could trap enough dust to reproduce the millimeter emission of a bright transition disk, while its cavity size is independently driven by photoevaporative dispersal. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.09687-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.09687) | **The Cygnus Allscale Survey of Chemistry and Dynamical Environments:  CASCADE. II. A detailed kinematic analysis of the DR21 Main outflow**  |
|| I. M. Skretas, et al. -- incl., <mark>H. Beuther</mark>, <mark>S. Li</mark> |
|*Appeared on*| *2023-09-19*|
|*Comments*| *26 pages, 37 figures. Accepted for publication in A&A*|
|**Abstract**| Molecular outflows are believed to be a key ingredient in the process of star formation. The molecular outflow associated with DR21 Main in Cygnus-X is one of the most extreme, in mass and size, molecular outflows in the Milky Way. The outflow is suggested to belong to a rare class of explosive outflows which are formed by the disintegration of protostellar systems.We aim to explore the morphology, kinematics,and energetics of the DR21 Main outflow, and compare those properties to confirmed explosive outflows to unravel the underlying driving mechanism behind DR21. Line and continuum emission are studied at a wavelength of 3.6\,mm with IRAM 30 m and NOEMA telescopes as part of the Cygnus Allscale Survey of Chemistry and Dynamical Environments (CASCADE) program. The spectra include ($J= 1-0$) transitions of HCO$^+$, HCN, HNC, N$_2$H$^+$, H$_2$CO, CCH tracing different temperature and density regimes of the outflowing gas at high-velocity resolution ($\sim$ 0.8 km s$^{-1}$). The map encompasses the entire DR21 Main outflow and covers all spatial scales down to a resolution of ~3" ($\sim$ 0.02 pc). Integrated intensity maps of the HCO$^+$ emission reveal a strongly collimated bipolar outflow with significant overlap of the blue- and red-shifted emission. The opening angles of both outflow lobes decrease with velocity, from $\sim80$ to 20$^{\circ}$ for the velocity range from 5 to 45 km s$^{-1}$ relative to the source velocity. No evidence is found for the presence of elongated, "filament-like" structures expected in explosive outflows. N$_2$H$^+$ emission near the western outflow lobe reveals the presence of a dense molecular structure which appears to be interacting with the DR21 Main outflow. The overall morphology as well as the detailed kinematics of the DR21 Main outflow is more consistent with that of a typical bipolar outflow instead of an explosive counterpart. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.09414-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.09414) | **The triggering process of an X-class solar flare on a small quadrupolar  active region**  |
|| Q. Song, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2023-09-19*|
|*Comments*| *24 pages, 7 figures, accepted for publication in ApJ*|
|**Abstract**| The occurrence of X-class solar flares and their potential impact on the space weather often receive great attention than other flares. But predicting when and where an X-class flare will occur is still a challenge. With the multi-wavelength observation from the Solar Dynamics Observatory and FengYun- 3E satellite, we investigate the triggering of a GOES X1.0 flare occurring in the NOAA active region (AR) 12887. Our results show that this unique X-class flare is bred in a relatively small but complex quadrupolar AR. Before the X-class flare, two filaments (F1 and F2) exist below a null-point topology of the quadrupolar AR. Magnetic field extrapolation and observation reveal that F1 and F2 correspond to two magnetic flux ropes with the same chirality and their adjacent feet rooted at nonconjugated opposite polarities, respectively. Interestingly, these two polarities collide rapidly, accompanied by photospheric magnetic flux emergence, cancellation and shear motion in the AR center. Above this site, F1 and F2 subsequently intersect and merge to a longer filament (F3) via a tether-cutting-like reconnection process. As a result, the F3 rises and erupts, involving the large-scale arcades overlying filament and the quadrupolar magnetic field above the AR, and eventually leads to the eruption of the X-class flare with a quasi-X-shaped flare ribbon and a coronal mass ejection. It suggests that the rapid collision of nonconjugated opposite polarities provides a key condition for the triggering of this X-class flare, and also provides a featured case for flare trigger mechanism and space weather forecasting. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2309.08752.md
    + _build/html/tmp_2309.08752/./SecondaryPlots/Agap_MassIntensity.png
    + _build/html/tmp_2309.08752/./SecondaryPlots/Rgap_MassIntensity.png
    + _build/html/tmp_2309.08752/./SecondaryPlots/Lx_MassIntensity.png
exported in  _build/html/2309.09687.md
    + _build/html/tmp_2309.09687/./correlations/Fco_Menv_correlation_plot.png
    + _build/html/tmp_2309.09687/./correlations/Fco_Lbol_correlation_plot.png
    + _build/html/tmp_2309.09687/./Appendix1/2.2um_UKIDSS_WFCAM_withhcn_hnc_h2cocontours.png
    + _build/html/tmp_2309.09687/./Images1/HCO+_H2CO_N2H+_intensities_across_line.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\revision}[1]{#1}$
$\newcommand{\revisionStrike}[1]{\sout{\textcolor{red}{#1}}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand{\St}{\ensuremath{\mathrm{St}}\xspace}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\matias}[1]{\textcolor{blue}{\textbf{Matias:} #1}}$
$\newcommand{\paola}[1]{\textcolor{pink}{\textbf{Paola:} #1}}$
$\newcommand{\til}[1]{\textcolor{red}{\textbf{Til:} #1}}$
$\newcommand{\barbara}[1]{\textcolor{red}{\textbf{Barbara:} #1}}$
$\newcommand{\sean}[1]{\textcolor{teal}{\textbf{Sean:} #1}}$
$\newcommand{\ana}[1]{\textcolor{teal}{\textbf{Ana:} #1}}$
$\newcommand{\sebastian}[1]{\textcolor{purple}{\textbf{Sebastian:} #1}}$
$\newcommand{\raphael}[1]{\textcolor{purple}{\textbf{Raphael:} #1}}$
$\newcommand{\giovanni}[1]{\textcolor{purple}{\textbf{Giovanni:} #1}}$
$\newcommand{\chapterautorefname}{Chapter}$
$\newcommand{\sectionautorefname}{Section}$
$\newcommand{\subsectionautorefname}{Section}$
$\newcommand{\subsubsectionautorefname}{Section}$
$\newcommand{\figureautorefname}{Figure}$</div>



<div id="title">

# Millimeter emission in photoevaporating disks is determined by early substructures

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2309.08752-b31b1b.svg)](https://arxiv.org/abs/2309.08752)<mark>Appeared on: 2023-09-19</mark> -  _Accepted for publication in A&A_

</div>
<div id="authors">

<mark>M. Gárate</mark>, et al.

</div>
<div id="abstract">

**Abstract:** Photoevaporation and dust-trapping are individually considered to be important mechanisms in the evolution and morphology of protoplanetary disks. However, it is not yet clear what kind of observational features are expected when both processes operate simultaneously. We studied how the presence (or absence) of early substructures, such as the gaps caused by planets, affects the evolution of the dust distribution and flux in the millimeter continuum of disks that are undergoing photoevaporative dispersal. We also tested if the predicted properties resemble those observed in the population of transition disks. We used the numerical code \texttt{Dustpy} to simulate disk evolution considering gas accretion, dust growth, dust-trapping at substructures, and mass loss due to X-ray and EUV (XEUV) photoevaporation and dust entrainment. Then, we compared how the dust mass and millimeter flux evolve for different disk models. We find that, during photoevaporative dispersal, disks with primordial substructures retain more dust and are brighter in the millimeter continuum than disks without early substructures, regardless of the photoevaporative cavity size.   Once the photoevaporative cavity opens, the estimated fluxes for the disk models that are initially structured are comparable to those found in the bright transition disk population ( $F_\textrm{mm} > 30  \textrm{mJy}$ ), while the disk models that are initially smooth have fluxes comparable to the transition disks from the faint population ( $F_\textrm{mm} < 30  \textrm{mJy}$ ), suggesting a link between each model and population. Our models indicate that the efficiency of the dust trapping determines the millimeter flux of the disk, while the gas loss due to photoevaporation controls the formation and expansion of a cavity, decoupling the mechanisms responsible for each feature.   In consequence, even a planet with a mass comparable to Saturn could trap enough dust to reproduce the millimeter emission of a bright transition disk, while its cavity size is independently driven by photoevaporative dispersal.

</div>

<div id="div_fig1">

<img src="tmp_2309.08752/./SecondaryPlots/Agap_MassIntensity.png" alt="Fig9" width="100%"/>

**Figure 9. -** Same as \autoref{Fig_LxParameter}, but for structured disks with different gap amplitudes, with a fixed location at $r_\textrm{gap}=\SI{40}{AU}$ and X-ray luminosity of $L_x = \SI{e30}{erg  s^{-1}}$. Notice that the axis scales are different from \autoref{Fig_LxParameter}.
  (*Fig_AgapParameter*)

</div>
<div id="div_fig2">

<img src="tmp_2309.08752/./SecondaryPlots/Rgap_MassIntensity.png" alt="Fig10" width="100%"/>

**Figure 10. -** Same as \autoref{Fig_LxParameter}, but for structured disks with different gap locations, with a fixed amplitude of $A_\textrm{gap}=4$ and X-ray luminosity of $L_x = \SI{e30}{erg  s^{-1}}$.
 Notice that the y-axis scale is different from \autoref{Fig_AgapParameter}.
  (*Fig_RgapParameter*)

</div>
<div id="div_fig3">

<img src="tmp_2309.08752/./SecondaryPlots/Lx_MassIntensity.png" alt="Fig8" width="100%"/>

**Figure 8. -** Evolution of the dust mass (_top_) and disk flux at $\lambda = \SI{1.3}{mm}$(_bottom_, assuming a distance of \SI{140}{pc}), for different X-ray luminosities $L_x$, with the black line corresponding to the fiducial value.
 The markers indicate the moment when photoevaporation opens a cavity in the inner disk (\sgquote{+} for the smooth disk, \sgquote{x} for the structured disk).
  (*Fig_LxParameter*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2309.08752"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\arraystretch}{1.5}$</div>



<div id="title">

# The Cygnus Allscale Survey of Chemistry and Dynamical Environments: CASCADE. II. A detailed kinematic analysis of the DR21 Main outflow

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2309.09687-b31b1b.svg)](https://arxiv.org/abs/2309.09687)<mark>Appeared on: 2023-09-19</mark> -  _26 pages, 37 figures. Accepted for publication in A&A_

</div>
<div id="authors">

I. M. Skretas, et al. -- incl., <mark>H. Beuther</mark>, <mark>S. Li</mark>

</div>
<div id="abstract">

**Abstract:** Molecular outflows are believed to be a key ingredient in the process of star formation. The molecular outflow associated with DR21 Main in Cygnus-X is one of the most extreme, in mass and size, molecular outflows in the Milky Way. The outflow is suggested to belong to a rare class of explosive outflows which are formed by the disintegration of protostellar systems. We aim to explore the morphology, kinematics,and energetics of the DR21 Main outflow, and compare thoseproperties to confirmed explosive outflows to unravel theunderlying driving mechanism behind DR21. Line and continuum emission are studied at a wavelength of 3.6 mm with IRAM 30 m and NOEMA telescopes as part of the Cygnus Allscale Survey of Chemistry and Dynamical Environments (CASCADE) program. The spectra include ( $J= 1-0$ ) transitions of HCO $^+$ , HCN, HNC, $N_2$ H $^+$ , $H_2$ CO, CCH (among others) tracing different temperature and density regimes of the outflowing gas at high-velocity resolution ( $\sim$ 0.8 km s $^{-1}$ ). The map encompasses the entire DR21 Main outflow and covers all spatial scales down to a resolution of  3 $\arcsec$ ( $\sim$ 0.02 pc). Integrated intensity maps of the HCO $^+$ emission reveal a strongly collimated bipolar outflow with significant overlap of the blue- and red-shifted emission. The opening angles of both outflow lobes decrease with velocity, from $\sim80$ to 20 $^{\circ}$ for the velocity range from 5 to 45 km s $^{-1}$ relative to the source velocity. No evidence is found for the presence of elongated, $\lq\lq$ filament-like $"$ structures expected in explosive outflows.$N_2$ H $^+$ emission near the western outflow lobe reveals the presence of a dense molecular structure which appears to be interacting with the DR21 Main outflow. The overall morphology as well as the detailed kinematics of the DR21 Main outflow is more consistent with that of a typical bipolar outflow instead of an explosive counterpart.

</div>

<div id="div_fig1">

<img src="tmp_2309.09687/./correlations/Fco_Menv_correlation_plot.png" alt="Fig15.1" width="50%"/><img src="tmp_2309.09687/./correlations/Fco_Lbol_correlation_plot.png" alt="Fig15.2" width="50%"/>

**Figure 15. -**  Outflow force over the envelope mass of the driving source for various protostellar sources. Right facing triangles represent low-mass sources from \citet{Mottram2017}, left facing triangles mark sources taken from \citet{Yildiz2015} and upwards are from \citet{vdm2013} while in blue are the Class I sources and in red the Class 0. Black diamonds mark intermediate mass sources \citep{vankempen2009}, grey squares mark high mass sources from \citet{Maud2015}, green \lq\lq$\times$" mark high mass sources from \citet{Beuther2002} and black stars mark high mass sources in Cygnus \citep{Skretas2022}. The red crosses mark a sample of high-mass 70$\mu$m dark sources \citep{Shanghuo2020}. The Cyan star marks G5.89-0.39, the magenta one marks Orion KL and the green one represents the DR21 Main outflow. The dashed, black line shows the best fit to the outflow force - envelope mass correlation for all sources, while red and grey show the best fits for the low- and high-mass sources respectively. (*fig:forcemass*)

</div>
<div id="div_fig2">

<img src="tmp_2309.09687/./Appendix1/2.2um_UKIDSS_WFCAM_withhcn_hnc_h2cocontours.png" alt="Fig16" width="100%"/>

**Figure 16. -** _UKIRT_/WFCAM continuum image of the DR21 Main region at 2.2 $\mu$m and the line emission in key gas tracers observed as part of CASCADE. White contours mark the 5$\sigma$ HCN (left), $H_2$CO (middle) and HNC (right) emission. (*fig:appcontours1*)

</div>
<div id="div_fig3">

<img src="tmp_2309.09687/./Images1/HCO+_H2CO_N2H+_intensities_across_line.png" alt="Fig7" width="100%"/>

**Figure 7. -** Average integrated intensities of HCO$^+$(in blue), $H_2$CO (in red), and $N_2$H$^+$(in green) across the interaction region in the western lobe of DR21. Intensities are integrated from $-$70 to 70, $-$20 to 20 and $-$20 to 10 km s$^{-1}$ for HCO$^+$, $H_2$CO and $N_2$H$^+$, respectively. The x-axis shows the distance in arcseconds, covering the extent of the relevant region where the outflow interacts with a dense structure (marked also in Fig. \ref{fig:interactioncontours}). The orange rectangle shows the area actively affected by the interaction. The intensities for $H_2$CO and $N_2$H$^+$ are scaled up by a factor of 8 in order for their distributions to be more easily comparable. (*fig:intens_across_line*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2309.09687"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

273  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

7  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
